<h1>Fake News Classifier Using LSTM</h1><br>
Dataset: https://www.kaggle.com/c/fake-news/data#

In [3]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/LSTM/fake-news/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
#drop nan values
df=df.dropna()

In [5]:
#get independend features
X=df.drop('label',axis=1)
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [6]:
y=df['label']
y.head(3)

0    1
1    0
2    1
Name: label, dtype: int64

In [7]:
X.shape

(18285, 4)

In [8]:
y.shape

(18285,)

In [9]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [10]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [11]:
##vocabulary size 
voc_size=5000

ONE HOT REPRESENTATION

In [12]:
#the X df is copied to messahe
message=X.copy()
message['title'][5]

'Jackie Mason: Hollywood Would Love Trump if He Bombed North Korea over Lack of Trans Bathrooms (Exclusive Video) - Breitbart'

In [13]:
message.reset_index(inplace=True)
#the nan values are removed to reset index use the above code

In [14]:
import nltk
import re
from nltk.corpus import stopwords

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

<h1>Data Preprocessing</h1>

In [16]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(message)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', message['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [17]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

<h1>Doing one hot encoding </h1><br>

In [18]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr
# print(len(onehot_repr))

[[4586, 1219, 3317, 2746, 3678, 4096, 1207, 127, 4010, 4727],
 [3745, 1665, 813, 4266, 3137, 2133, 3749],
 [2634, 1301, 2610, 4700],
 [629, 2738, 2544, 1842, 1064, 2018],
 [82, 3137, 2043, 4805, 297, 3710, 3137, 1509, 1319, 4133],
 [2020,
  3591,
  2061,
  4466,
  4656,
  3971,
  1635,
  1113,
  2860,
  3866,
  2098,
  3719,
  1183,
  4178,
  3749],
 [1260, 1871, 3385, 295, 3341, 2933, 3549, 4853, 3346, 648, 249],
 [2855, 1689, 2422, 2158, 3935, 3038, 3971, 4307, 3346, 648, 249],
 [230, 2005, 1528, 2314, 4054, 1626, 2440, 4818, 3971, 1173],
 [20, 1313, 3821, 4257, 4815, 4551, 1039, 4310],
 [3514, 4550, 3811, 1647, 3682, 315, 1895, 3704, 2547, 2169, 3018],
 [1842, 3970, 3678, 1626, 3971, 3935],
 [2700, 3080, 435, 3734, 1278, 3719, 2778, 4869, 1683],
 [4905, 1623, 2923, 1712, 4554, 1860, 2683, 3346, 648, 249],
 [2287, 1497, 3111, 798, 2805, 3346, 648, 249],
 [1967, 920, 2295, 998, 571, 3650, 3909, 3862, 3917, 2294],
 [995, 1806, 1665],
 [1151, 1684, 2903, 4036, 3971, 3465, 4963, 3749],
 

Embedding Representation

In [19]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)
print(embedded_docs.shape)
print(embedded_docs[1])

[[   0    0    0 ...  127 4010 4727]
 [   0    0    0 ... 3137 2133 3749]
 [   0    0    0 ... 1301 2610 4700]
 ...
 [   0    0    0 ... 3346  648  249]
 [   0    0    0 ... 1465 3565  318]
 [   0    0    0 ... 4975  182 2361]]
(18285, 20)
[   0    0    0    0    0    0    0    0    0    0    0    0    0 3745
 1665  813 4266 3137 2133 3749]


In [20]:
#model
embedding_vector_feature=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_feature,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
len(embedded_docs),y.shape

(18285, (18285,))

In [22]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y) 

In [23]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [24]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)


<h1>MODEL TRAINING</h1>

In [25]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 7s 34ms/step - loss: 0.3291 - accuracy: 0.8450 - val_loss: 0.1929 - val_accuracy: 0.9165
Epoch 2/10
192/192 [==============================] - 6s 32ms/step - loss: 0.1382 - accuracy: 0.9467 - val_loss: 0.1993 - val_accuracy: 0.9157
Epoch 3/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0964 - accuracy: 0.9651 - val_loss: 0.2205 - val_accuracy: 0.9065
Epoch 4/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0695 - accuracy: 0.9756 - val_loss: 0.2836 - val_accuracy: 0.9143
Epoch 5/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0466 - accuracy: 0.9840 - val_loss: 0.3059 - val_accuracy: 0.9142
Epoch 6/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0337 - accuracy: 0.9892 - val_loss: 0.3912 - val_accuracy: 0.9114
Epoch 7/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0202 - accuracy: 0.9932 - val_loss: 0.3899 - val_accuracy: 0.9090

In [26]:
#Adding Dropout: with hyperparmeter tunning
# from tensorflow.keras.layers import Dropout
# ## Creating model
# embedding_vector_features=40
# model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(Dropout(0.3))
# model.add(LSTM(100))
# model.add(Dropout(0.3))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Perfomance metrix 

In [27]:
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [28]:
from sklearn.metrics import confusion_matrix

In [29]:
confusion_matrix(y_test,y_pred)

array([[3136,  283],
       [ 263, 2353]])

In [30]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9095277547638774